# Luad Hemostasis Circle Heatmap

This notebooks takes the genes that are a hit from the Luad Reactome_2016 UPAR pathway and maps them on a large circle heat map. 

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats
import re
import sys 
import statsmodels.stats.multitest

import gseapy as gp
from gseapy.plot import barplot, dotplot

import cptac
import cptac.utils as u
import plot_utils as p 

/Users/Lindsey/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


Load df with all of the genes that are FDR significant. Then get list of just the gene names and use them to run a GSEA. 

In [4]:
#Get append version of the df with all cancer type, fdr sig trans results
df_FDR_append = pd.read_csv("../Step3.2_combining_pearson_dfs/csv_files/pancan_EGFR_pearson_all_prot_append_return_all.csv")
df_FDR_append = df_FDR_append.drop(['Unnamed: 0'], axis=1)


In [5]:
luad_FDR = pd.read_csv("../Step3.1_Pearson_dfs_by_cancer/csv_files/Luad_EGFR_all_pearson_FDR.csv")

In [12]:
df_FDR= luad_FDR.drop(['Unnamed: 0'], axis=1)
df_FDR = df_FDR.set_index("Comparison")
df1_transposed = df_FDR.T 


In [7]:
luad_prot = df1_transposed.columns.values.tolist()
luad_genes = []
for gene in luad_prot :
    luad_genes.append((re.sub("_proteomics", "", gene)))
len(luad_genes)

1198

In [13]:
luad_enr = gp.enrichr(gene_list = luad_genes, description='Tumor_partition', gene_sets='Reactome_2016', 
                       outdir='test/enrichr_kegg')
luad_enr.res2d.head(4)

,Gene_set,Term,Overlap,P-value,Adjusted P-value,Old P-value,Old Adjusted P-value,Odds Ratio,Combined Score,Genes
0,Reactome_2016,Immune System Homo sapiens R-HSA-168256,158/1547,1.186818e-11,1.815831e-08,0,0,1.705061,42.894496,ATF1;CYFIP2;APP;AHCYL1;NCF1;WIPF1;PROS1;WIPF2;...
1,Reactome_2016,Innate Immune System Homo sapiens R-HSA-168249,93/807,8.043351e-10,6.153164e-07,0,0,1.923900,40.288409,ATF1;CYFIP2;APP;AHCYL1;CDKN1B;WIPF1;WIPF2;PROS...
2,Reactome_2016,RHO GTPases Activate WASPs and WAVEs Homo sapi...,14/36,7.762482e-09,3.958866e-06,0,0,6.492302,121.237012,CYFIP2;WIPF1;WIPF2;WAS;BRK1;ARPC4;ARPC5;PTK2;A...
3,Reactome_2016,Hemostasis Homo sapiens R-HSA-109582,66/552,6.232478e-08,2.383923e-05,0,0,1.996080,33.116784,APP;DOCK5;DGKB;PROS1;F13A1;PIK3CD;ARRB2;CLU;AK...


In [9]:
#get just the upa genes
luad_df = luad_enr.res2d
genes = luad_df.iloc[3,9]
genes = genes.split(';')

genes.remove("GRB2")

len(genes)


65

In [14]:
#filter down df with just upa genes 
pathway_names = []
for gene in genes:
    gene += "_proteomics"
    pathway_names.append(gene)

df_FDR_pathway = df_FDR_append[df_FDR_append.Comparison.isin(pathway_names)]


In [11]:
#Make plot using plot utils
p.plotCircleHeatMap(df_FDR_pathway, "P_value","Correlation","Comparison","Cancer Type",plot_width= 1000, plot_height = 650)

/Users/Lindsey/anaconda3/lib/python3.7/site-packages/plot_utils/__init__.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["size2"] = df[circle_var].apply(lambda x: -1*(np.log(x)))
/Users/Lindsey/anaconda3/lib/python3.7/site-packages/plot_utils/__init__.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['size'] = (df["size2"])*3


Loading BokehJS ...